## File search assistant
The McKessonMDTranscripts demonstrates the creation and execution of an assistant using the file_search feature with a knowledge base based on earnings call transcripts downloaded from the internet.

## First steps
- First, download AntRunLib from Nuget
- At least once, setup the enviroment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.6.5"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAIConfigFactory.Get();
var client = GetOpenAIClient(config);


Installed Packages AntRunnerLib, 0.6.5

## Ensure the McKessonMDTranscripts assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\McKessonMDTranscripts" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** enviroment variable.

"McKessonMDTranscripts" contains the following files:
```
│   manifest.json
│   prompt.md
│
└───VectorStores
    └───McKessonMDTranscripts
            211101-MCK-Q2FY22-Earnings-Call-Transcript.md
            220202-MCK-Q3FY22-Earnings-Call-Transcript.md
            220505-MCK-Q4FY22-Earnings-Call-Transcript.md
            220803-MCK-Q1FY23-Earnings-Call-Transcript.md
            230508-MCK-Q4FY23-Earnings-Call-Transcript.md
            230802-MCK-Q1FY24-Earnings-Call-Transcript.md
            231101-MCK-Q2FY24-Earnings-Call-Transcript.md
            MCK-Q1FY21-Transcript.md
            MCK-Q1FY22-Transcript.md
            MCK-Q2-FY23-Earnings-Transcript.md
            MCK-Q2FY21-Transcript.md
            MCK-Q3-FY24-Earnings-Transcript.md
            MCK-Q4-FY24-Earnings-Call-Transcript.md
            MCK-Q4FY21-Transcript.md
            MCK-US-20230201-2761065-C.md
            Q1-FY19-Earnings-Call-Transcript.md
            Q1-FY20-Earnings-Call-Transcript.md
            Q2-FY19-Earnings-Call-Transcript.md
            Q2-FY20-Earnings-Call-Transcript.md
            Q3-FY19-Earnings-Call-Transcript.md
            Q3FY20-MCK-Earnings-Call-Transcript.md
            Q3FY21-MCK-Earnings-Call-Transcript_FINAL.md
            Q4-FY19-Earnings-Call-Transcript.md
            Q4FY20-MCK-Corrected-Transcript.md            
```        

### Explanation
AssistantUtility.Create will create the vector store if necessary, upload the files, and then update the manifest with the vector store ID before creating the assistant.

In [2]:
var assistantId = await AssistantUtility.Create("McKessonMDTranscripts", config);
Console.WriteLine(assistantId)

asst_WZ1DNQ9npBTjzoSR2fA8GpUp


## Run the Assistant

output.Dialog shows the conversation. 

In [3]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "McKessonMDTranscripts",
    Instructions = "How did covid impact results in Q3 2022?",
    UseConversationEvaluator = false
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
Console.WriteLine(output.Dialog)


User: How did covid impact results in Q3 2022?

Assistant: I searched my knowledge base for the answer.

In Q3 2022, McKesson's results were significantly impacted by the COVID-19 pandemic and the emergence of the Omicron variant. The company observed a non-linear recovery trend throughout the quarter. Initially, volume and utilization trends were recovering as COVID-19 cases declined, but the unexpected emergence and spread of the Omicron variant in December led to a renewed impact on the business. Despite these challenges, McKesson demonstrated resilience and adaptability, continuing to support their customers and communities effectively【4:4†source】【4:4†source】.



In [4]:
Console.WriteLine(output.LastMessage)

In Q3 2022, McKesson's results were significantly impacted by the COVID-19 pandemic and the emergence of the Omicron variant. The company observed a non-linear recovery trend throughout the quarter. Initially, volume and utilization trends were recovering as COVID-19 cases declined, but the unexpected emergence and spread of the Omicron variant in December led to a renewed impact on the business. Despite these challenges, McKesson demonstrated resilience and adaptability, continuing to support their customers and communities effectively【4:4†source】【4:4†source】.


## Annotations

In [5]:
foreach(var annotation in output.Annotations)
{
    Console.WriteLine(annotation.FileCitation.FileId);
}

assistant-fjWqtpdCyNecNYDdGK9vhrG4
assistant-fjWqtpdCyNecNYDdGK9vhrG4
